The `asyncio` module provides tools for building concurrent applications using coroutines. While the threading module implements concurrency through application threads, and multiprocessing implements concurrency using system processes, asyncio uses a single-threaded, single-process approach in which parts of an application cooperate to switch tasks explicitly at optimal times. Most oftenly this context switching occurs when the program would otherwise block waiting to read or write data, but asyncio also includes support for scheduling code to run at a specific future time, to enable one coroutine to wait for another to complete, for handling system signals, and for recognizing other events that may be reasons for an application to change what it is working on. `Async I/O` is a single-threaded, single-process design: it uses cooperative multitasking for performing concurrent execution of code.

In [23]:
import asyncio

In [24]:
async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')
    return "YES"

In [25]:
main()

<coroutine object main at 0x000001CF3C6FB640>

In [27]:
loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(main())
except RuntimeError as e:
    print(e.__class__, e)

<class 'RuntimeError'> This event loop is already running


C:\Users\admin\AppData\Local\Temp\ipykernel_11380\3827710070.py:5: RuntimeWarning: coroutine 'main' was never awaited
  print(e.__class__, e)


In [28]:
await main()

Hello ...
... World!


'YES'

To actually run a coroutine, asyncio provides three main mechanisms:

- The asyncio.run() function to run the top-level entry point function
- Awaiting on a coroutine
- The asyncio.create_task() function to run coroutines concurrently as asyncio Tasks

We say that an object is an awaitable object if it can be used in an await expression.

There are three main types of `awaitable` objects: `coroutines`, `Tasks`, and `Futures`. The term `coroutine` can be used for two closely related concepts:

- a coroutine function: an async def function;

- a coroutine object: an object returned by calling a coroutine function.

In [29]:
async def temp(text):
    return f'We get {text}'

In [30]:
temp('hello')

<coroutine object temp at 0x000001CF3C6FA140>

In [31]:
async def inner(y):
    print(f'Inner - {y}')
    return 'Yes'

async def outer(x):
    print(f'Outter - {x}')
    inner_x = await inner(x)
    print(inner_x)
    

In [32]:
await outer('hello')

Outter - hello
Inner - hello
Yes


In [33]:
task = asyncio.create_task(temp('task 1'))

task2 = asyncio.ensure_future(temp("task 2"))

In [34]:
task2

<Task finished name='Task-10' coro=<temp() done, defined at C:\Users\admin\AppData\Local\Temp\ipykernel_11380\3204015853.py:1> result='We get task 2'>

In [35]:
task

<Task finished name='Task-9' coro=<temp() done, defined at C:\Users\admin\AppData\Local\Temp\ipykernel_11380\3204015853.py:1> result='We get task 1'>

In [36]:
task.done()

True

In [37]:
task.result()

'We get task 1'

In [38]:
def handle():
    print('Task is done')

In [39]:
task.cancel()

False

In [40]:
async def input_number():
    return input('Number: ')

In [41]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        await input_number()
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    

In [42]:
async def gather_tasks():
    await asyncio.gather(
        factorial("A", 4),
        factorial("B", 3),
        factorial("C", 2),
    )

In [43]:
await gather_tasks()

Task A: Compute factorial(2)...
Task A: Compute factorial(3)...
Task A: Compute factorial(4)...
Task A: factorial(4) = 24
Task B: Compute factorial(2)...
Task B: Compute factorial(3)...
Task B: factorial(3) = 6
Task C: Compute factorial(2)...
Task C: factorial(2) = 2


In [60]:
import aiohttp
import os

In [61]:
session = aiohttp.ClientSession()
async def future_example(session, url):
    resp = await session.get(url)
    data = await resp.read()
    filename = os.path.basename(url)
    with open(filename, 'wb') as image_file:
        image_file.write(data)
        print(f'{filename} was downloaded...')
    return resp, data

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001CF3D0B64D0>


In [62]:
resp, data = await future_example(session)

TypeError: future_example() missing 1 required positional argument: 'url'

In [52]:
data

NameError: name 'data' is not defined

In [53]:
resp

NameError: name 'resp' is not defined

In [57]:
url = ['https://upload.wikimedia.org/wikipedia/commons/9/9d/Python_bivittatus_1701.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/4/48/Python_Regius.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/d/d3/Baby_carpet_python_caudal_luring.jpg']

In [58]:
async def gather_tasks():
    await asyncio.gather(
        future_example(session, url[0]),
        future_example(session, url[1]),
        future_example(session, url[2]),
    )

In [59]:
await gather_tasks()

Python_Regius.jpg was downloaded...
Baby_carpet_python_caudal_luring.jpg was downloaded...
Python_bivittatus_1701.jpg was downloaded...
